In [8]:
# Import the modules
import math
import numpy as np
import itertools
import pandas as pd

# Define the array of lambda values (adjust as needed)
lambdas = [5.90e-2, 1e-2]

# Generate all the binary combinations of success and failure for two events
sequences = list(itertools.product([0, 1], repeat=len(lambdas)))

# Initialize an array to hold the probabilities of each sequence
probabilities = np.zeros(len(sequences))

# Ask the user to enter the initiating event frequency value
ief = float(input("Enter the initiating event frequency value: "))

# Define the function for calculating F (product of event values)
def calculate_F(events):
    result = 1.0  # Initialize the result to 1.0

    for event in events:
        result *= event  # Multiply each event value

    return result  # Return the product of all event values

# Loop over all the sequences and calculate the probability of each one using calculate_F
for i, seq in enumerate(sequences):
    # Initialize a list to hold the lambda values for each event in the sequence
    events = list(lambdas)

    # If there are no failures in the sequence, set the probability to zero
    if 0 not in seq:
        probabilities[i] = 0
    else:
        # If there is only one failure in the sequence, set the probability to ief * lambda
        if seq.count(0) == 1:
            index_of_failure = seq.index(0)
            probabilities[i] = ief * events[index_of_failure]

        # If there are more than one failures in the sequence, set the probability to ief * lambda-top * (1 - lambda-top)^(n-1) where n is number of successes
        else:
            lambdatop = calculate_F([l for j, l in enumerate(events) if seq[j] == 0])
            probabilities[i] = ief * lambdatop * (1 - lambdatop)**(len(seq) - seq.count(0) - 1)

# Convert the sequences and probabilities to lists of strings
outcomes = [''.join(['S' if x == 1 else 'F' for x in seq]) for seq in sequences]
probabilities = [f"{x:.1e}" for x in probabilities]

# Create a pandas dataframe with the outcomes and probabilities columns
df = pd.DataFrame({'Outcome': outcomes, 'Probability': probabilities})

# Print the dataframe as a table
print(df.to_string(index=False))

# Ask the user if they want to exclude or include some sequences in the calculation
answer = input("Do you want to exclude or include some sequences in the calculation? (exclude/include/n) ")

# Initialize lists to hold the excluded and included sequences
excluded = []
included = []

# If the user answers exclude, ask them to enter the sequences they want to exclude until they say done
if answer.lower() == 'exclude':
    while True:
        sequence = input("Enter a sequence you want to exclude (e.g. S, S) or type done to finish: ")
        if sequence.lower() == 'done':
            break
        elif sequence in outcomes:
            excluded.append(sequence)
            print(f"Sequence {sequence} is excluded.")
        else:
            print(f"Sequence {sequence} is not valid. Please enter a valid sequence.")

    # Filter out the excluded sequences from the dataframe
    df_filtered = df[~df['Outcome'].isin(excluded)]
    cdf = df_filtered['Probability'].astype(float).sum()

    # Print the filtered dataframe and updated list as a table
    print("The updated list after excluding sequences is:")
    print(df_filtered.to_string(index=False))

    # Print the core damage frequency as the sum of the remaining probabilities
    print(f"Core Damage Frequency is {cdf:.1e}")

# If the user answers include, ask them to enter the sequences they want to include until they say done
elif answer.lower() == 'include':
    while True:
        sequence = input("Enter a sequence you want to include (e.g. S, S) or type done to finish: ")
        if sequence.lower() == 'done':
            break
        elif sequence in outcomes:
            included.append(sequence)
            print(f"Sequence {sequence} is included.")
        else:
            print(f"Sequence {sequence} is not valid. Please enter a valid sequence.")

    # Filter out the included sequences from the dataframe
    df_filtered = df[df['Outcome'].isin(included)]
    cdf = df_filtered['Probability'].astype(float).sum()

    # Print the filtered dataframe and updated list as a table
    print("The updated list after including sequences is:")
    print(df_filtered.to_string(index=False))

    # Print the core damage frequency as the sum of the remaining probabilities
    print(f"Core Damage Frequency is {cdf:.1e}")

# If the user answers neither exclude nor include, print an error message and exit the program
elif answer.lower() != 'n':
    print("Invalid input. Please enter 'exclude', 'include', or 'n'.")
    exit()


Enter the initiating event frequency value: 1e-2
Outcome Probability
     FF     5.9e-06
     FS     5.9e-04
     SF     1.0e-04
     SS     0.0e+00
Do you want to exclude or include some sequences in the calculation? (exclude/include/n) exclude
Enter a sequence you want to exclude (e.g. S, S) or type done to finish: SS
Sequence SS is excluded.
Enter a sequence you want to exclude (e.g. S, S) or type done to finish: done
The updated list after excluding sequences is:
Outcome Probability
     FF     5.9e-06
     FS     5.9e-04
     SF     1.0e-04
Core Damage Frequency is 7.0e-04
